# Weather Data Analysis Project - Programming for Data Analytics

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
# Filename & Data Directory
filename = "shannon_data.csv"
datadir = r"C:\Users\ZMH\OneDrive\Desktop\PFDA\pfda-project\\" # still can't get "../../data" working
fullpath = datadir + filename

# Read in CSV file
df = pd.read_csv(fullpath, skiprows=23)

df.head()

C:\Users\ZMH\AppData\Local\Temp\ipykernel_10112\4267985318.py:7: DtypeWarning: Columns (6,12,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fullpath, skiprows=23)


,date,ind,rain,ind.1,temp,ind.2,wetb,dewpt,vappr,rhum,...,ind.3,wdsp,ind.4,wddir,ww,w,sun,vis,clht,clamt
0,01-sep-1945 01:00,0,0.1,0,15.3,0,14.6,14.4,16.0,92,...,1,17,1,90,50,6,0.0,7000,16,8
1,01-sep-1945 02:00,2,0.0,0,15.0,0,14.6,14.4,16.4,95,...,1,17,1,90,51,5,0.0,7000,9,8
2,01-sep-1945 03:00,0,0.1,0,15.0,0,14.6,14.4,16.4,95,...,1,17,1,100,51,6,0.0,7000,9,8
3,01-sep-1945 04:00,2,0.0,0,15.0,0,14.4,13.8,16.0,94,...,1,14,1,90,20,5,0.0,7000,9,8
4,01-sep-1945 05:00,0,0.0,0,15.0,0,14.5,14.4,16.1,95,...,1,16,1,110,2,2,0.0,10000,17,7
